In [ ]:
# https://jakevdp.github.io/PythonDataScienceHandbook/05.09-principal-component-analysis.html
# https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60
# https://www.askpython.com/python/examples/principal-component-analysis
# https://www.datacamp.com/community/tutorials/principal-component-analysis-in-python

In [ ]:
# https://www.askpython.com/python/examples/principal-component-analysis
# Not much explaination of iris data example application.
# Importing required libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
import sklearn 
sklearn.__version__

In [ ]:
# Generate a more real world data set that needs normalizing

#X = load_iris()
X = load_iris().data
X_meaned = X - np.mean(X , axis = 0)
#X.data

In [ ]:
# Generate a dummy dataset.
X = np.random.randint(10,50,100).reshape(20,5) 
# mean Centering the data  
X_meaned = X - np.mean(X , axis = 0)

In [ ]:
pca = PCA(n_components=3)
pca.fit(X)


In [ ]:
X

In [ ]:
np.mean(X , axis = 0)

In [ ]:
X_meaned

In [ ]:
cov_mat = np.cov(X_meaned , rowvar = False)
cov_mat

In [ ]:
#Calculating Eigenvalues and Eigenvectors of the covariance matrix
eigen_values , eigen_vectors = np.linalg.eigh(cov_mat)
eigen_vectors

In [ ]:
eigen_values

In [ ]:
#sort the eigenvalues in descending order
sorted_index = np.argsort(eigen_values)[::-1]
sorted_index

In [ ]:
sorted_eigenvalue = eigen_values[sorted_index]
sorted_eigenvectors = eigen_vectors[:,sorted_index]
# Why does this reverse the oreder of the dimensions too?
sorted_eigenvectors

In [ ]:
# select the first n eigenvectors, n is desired dimension
# of our final reduced data.
 
n_components = 2 #you can select any number of components.
eigenvector_subset = sorted_eigenvectors[:,0:n_components]
eigenvector_subset

In [ ]:
# you can see these are Transposes of eigenvector_subset
# when using sklearn.decomposition.PCA, so I ready for the dot product below.

print(pca.components_)
# this seems to make no sense, variance should be less than 1
print(pca.explained_variance_ratio_)  

In [ ]:
# shows exp_variances of original data's PCA's for only the n you retained.
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');


In [ ]:
#Transform the data 
# to get data reduced to lower dimensions from higher dimensions.

X_reduced = np.dot(eigenvector_subset.transpose(),X_meaned.transpose()).transpose()
plt.scatter(X_reduced[:, 0], X_reduced[:, 1])
plt.show()

In [ ]:
print(eigenvector_subset.transpose())
print(X_meaned.transpose())
np.dot(eigenvector_subset.transpose(),X_meaned.transpose())

In [ ]:
# Normaly you would first scale and normalize the data and fit like this
X_std = StandardScaler().fit_transform(X) # normalizing the data 
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X_std)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
#from sklearn.datasets import load_iris
import sklearn 
print(sklearn.__version__)
print(sns.__version__)

In [ ]:
df_segmentation = pd.read_csv('segmentation data.csv', index_col=0)
df_segmentation.head()


In [ ]:
plt.figure(figsize=(12,9))
plt.scatter(df_segmentation.iloc[:,2],df_segmentation.iloc[:,4])
plt.xlabel('Age')
plt.ylabel('Income')
plt.title('Visualize raw age/income data')




In [ ]:
scaler = StandardScaler()
segmentation_std = scaler.fit_transform(df_segmentation)
segmentation_std[0]

In [ ]:
pca = PCA()
pca.fit(segmentation_std)
pca

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');


In [ ]:
pca = PCA(n_components=3)
pca.fit(segmentation_std)
scores_pca = pca.transform(segmentation_std)

In [ ]:
# 
wcss = []
for i in range(1,21):
    kmeans_pca = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans_pca.fit(scores_pca)
    wcss.append(kmeans_pca.inertia_)
    
    

In [ ]:
plt.figure(figsize = (10,8))
plt.plot(range(1,21),wcss, marker ='o', linestyle = '--')
plt.xlabel('number of clusters')
plt.ylabel('WCSS');
plt.title('k-means with pca clustering');
plt.show()

In [ ]:
kmeans_pca = KMeans(n_clusters = 4, init = 'k-means++', random_state = 42)
kmeans_pca.fit(scores_pca)


In [ ]:
df_seg_pca_kmeans = pd.concat([df_segmentation.reset_index(drop=True),
                               pd.DataFrame(scores_pca)],axis = 1)
df_seg_pca_kmeans.columns.values[-3:] = ['pc 1','pc 2','pc 3']
df_seg_pca_kmeans['Segment K-means PCA'] = kmeans_pca.labels_

df_seg_pca_kmeans.head()

In [ ]:
x_axis =  df_seg_pca_kmeans['pc 2']
y_axis =  df_seg_pca_kmeans['pc 1']
plt.figure(figsize = (10,8))
sns.scatterplot(x_axis,y_axis,hue =df_seg_pca_kmeans['Segment K-means PCA'], palette = ['g','r','c','m'] )
plt.title('Clusters by PCA components')
plt.show()

In [ ]:
x_axis =  df_seg_pca_kmeans['pc 3']
y_axis =  df_seg_pca_kmeans['pc 2']
plt.figure(figsize = (10,8))
sns.scatterplot(x_axis,y_axis,hue =df_seg_pca_kmeans['Segment K-means PCA'], palette = ['g','r','c','m'] )
plt.title('Clusters by PCA components 3,2')
plt.show()